In [1]:
import pandas as pd
import os
from datetime import date, datetime

In [ ]:
# Path do arquivo - Só muda caminho
caminho = r'C:\Users\A0161597\OneDrive - Telefonica\Documentos\PIR MOVEL\2025\04-ABRIL - Copia'
caminhopadrao = r"C:\Users\A0161597\OneDrive - Telefonica\Documentos\PIR MOVEL\2025\04-ABRIL"
arquivos = os.listdir(caminhopadrao)

sufixos_meses = {
    1: 'JAN',
    2: 'FEV',
    3: 'MAR',
    4: 'ABR',
    5: 'MAI',
    6: 'JUN',
    7: 'JUL',
    8: 'AGO',
    9: 'SET',
    10: 'OUT',
    11: 'NOV',
    12: 'DEZ'
}

data = date.today().strftime("%Y-%m-%d")
data = "2025-04-01"

data2 = date.today()
mes_numero = data2.month
ano_curto = data2.strftime('%y') 
sufixo_mes = sufixos_meses[mes_numero]
mes_redes_df = f"{sufixo_mes}{ano_curto}"
mes_redes_df = "ABR25"


In [3]:
# Le arquivo e lista nome planilhas
raiox_file = pd.ExcelFile(os.path.join(caminho, 'RAIOX.xlsb'))
raiox = raiox_file.sheet_names

deepdive_file = pd.ExcelFile(os.path.join(caminho, 'DEEPDIVE.xlsx'))
deepdive = deepdive_file.sheet_names

toro_file = pd.ExcelFile(os.path.join(caminho, 'TORO.xlsx'))
toro = toro_file.sheet_names

bi_file = pd.ExcelFile(os.path.join(caminho, 'TRAFEGOBI.xlsx'))
bi = bi_file.sheet_names

redes_file = pd.ExcelFile(os.path.join(caminho, 'TRAFEGOREDES.xlsx'))
redes = redes_file.sheet_names

padrao_file = pd.ExcelFile(os.path.join(caminhopadrao, '042025.xlsx'))
padrao = padrao_file.sheet_names

In [4]:
# Seleciona qual planilha

print(f'\nRaio X:\n{raiox}')
print(f'\nDeep Dive:\n{deepdive}')
print(f'\nToro:\n{toro}')
print(f'\nTráfego BI:\n{bi}')
print(f'\nTráfego Redes:\n{redes}')
print(f'\nPadrão:\n{padrao}')

raiox = raiox_file.sheet_names[0]
deepdive = deepdive_file.sheet_names[2]
toro = toro_file.sheet_names[0]
bi = bi_file.sheet_names[0]
redes = redes_file.sheet_names[1]
padrao = padrao_file.sheet_names[0]


Raio X:
['Planilha_Consol']

Deep Dive:
['Abr_25_Resumo', 'Graficos_PIR', 'DIN_PIR', 'Local_Cidades']

Toro:
['Consulta RF Abr25', 'Resumo_Abr25']

Tráfego BI:
['Planilha1']

Tráfego Redes:
['Resumo', 'Municípios']

Padrão:
['BASE', 'DEEP DIVE', 'TRAFEGO REDES', 'TRAFEGO BI', 'TRAFEGO BI W']


In [5]:
# Abre planilha

raiox_df = raiox_file.parse(raiox, header=5)
deepdive_df = deepdive_file.parse(deepdive, header=2)
toro_df = toro_file.parse(toro, header=1)
bi_df = bi_file.parse(bi)
redes_df = redes_file.parse(redes, header=4)
padrao_df = padrao_file.parse(padrao)

In [6]:
#Trata planilha TORO para join

padrao_df = padrao_df.iloc[:, 0:8]
padrao_df['ANO MÊS'] = data

toro_grouped = toro_df.loc[:, ['IBGE', '2G', '3G', '4G', '5G.1']]
toro_grouped = toro_grouped.groupby('IBGE')[['2G', '3G', '4G', '5G.1']].sum()
toro_grouped['TOTAL'] = toro_df.groupby('IBGE').size()
toro_grouped = toro_grouped[['TOTAL',  '5G.1']].reset_index()

toro_grouped.rename(columns={
    '5G.1': '5G'
}, inplace=True)

In [7]:
#Trata planilha REDES para join 

gg_redes = f'2G {mes_redes_df}' 
ggg_redes = f'3G {mes_redes_df}' 
gggg_redes = f'4G {mes_redes_df}' 
ggggg_redes = f'5G {mes_redes_df}' 

redes_grouped = redes_df.groupby('IBGE')[[gg_redes,ggg_redes,gggg_redes,ggggg_redes]].sum()
redes_grouped['TOTAL'] = redes_grouped[[gg_redes,ggg_redes,gggg_redes,ggggg_redes]].sum(axis=1)
redes_grouped = redes_grouped[['TOTAL', ggggg_redes]].reset_index()

In [8]:
#Trata planilha BI para join 

bi_df.rename(columns={
    'CD_IBGE_OK' : 'IBGE'
}, inplace=True)

bi_grouped = bi_df[~bi_df['DS_PRDT'].str.contains('PJ_M2M', na=False)]

bi_grouped_total = bi_grouped.groupby('IBGE')[['Soma de IN_PRQE_DADO_PGW', \
'Soma de QT_TRFG_PGW','Soma de VL_FTRM']].sum().reset_index()

bi_grouped_5g = bi_grouped[bi_grouped['DS_TCNL_RSMO'].str.contains('5G', na=False)]
bi_grouped_5g = bi_grouped_5g.groupby('IBGE')[['Soma de IN_PRQE_DADO_PGW',\
'Soma de QT_TRFG_PGW']].sum().reset_index()

bi_grouped_total.rename(columns={
    'Soma de IN_PRQE_DADO_PGW' : 'PARQUE BI TOTAL',
    'Soma de QT_TRFG_PGW' : 'TRAF. BI TOTAL MB',
    'Soma de VL_FTRM' : 'FATURAMENTO'
}, inplace=True)

bi_grouped_5g.rename(columns={
    'Soma de IN_PRQE_DADO_PGW' : 'PARQUE BI 5G',
    'Soma de QT_TRFG_PGW' : 'TRAF. BI 5G MB'
}, inplace=True)

bi_merged = pd.merge(bi_grouped_total, bi_grouped_5g, how='left', on='IBGE')

bi_merged_partei = bi_merged[['IBGE','TRAF. BI TOTAL MB','TRAF. BI 5G MB']]

bi_merged_parteii = bi_merged[['IBGE','PARQUE BI TOTAL','PARQUE BI 5G']]

bi_merged_parteiii = bi_merged[['IBGE','FATURAMENTO']]

In [ ]:
#Trata planilha DeepDive para join 

deepdive_df.rename(columns={
    'CODIGO_IBGE':'IBGE',
    'Soma de PARQUE':'PARQUE DESVIADO',
    'Soma de TRAFEGO_PGW_GB_':'TRAF DESVIADO',
    'Soma de RECEITA_TOTAL_LIQUIDA':'REC DESVIADO'
},inplace=True)

deepdive_grouped = deepdive_df[['IBGE', 'PARQUE DESVIADO',
'TRAF DESVIADO', 'REC DESVIADO']]

In [ ]:
#Cria e formata base

base_final = pd.merge(padrao_df, toro_grouped, how='left', left_on='IBGE', right_on='IBGE')

base_final.rename(columns={
    'TOTAL' : 'SITES',
    '5G' : 'SITES 5G'
}, inplace=True)

base_final = pd.merge(base_final, redes_grouped, how='left', left_on='IBGE', right_on='IBGE')

base_final.rename(columns={
    'TOTAL' : 'TRAF. REDES TOTAL MB',
    ggggg_redes : 'TRAF. REDES 5G MB'
}, inplace=True)

base_final = pd.merge(base_final, bi_merged_partei, how='left', left_on='IBGE', right_on='IBGE')

base_final['TRAF PREVISTO'] = ""
base_final['% TRAF PREVISTO'] = ""

base_final = pd.merge(base_final, deepdive_grouped, how='left', on='IBGE')

base_final['TRAF. REDES TOTAL GB'] = base_final['TRAF. REDES TOTAL MB']/1000
base_final['TRAF. REDES 5G GB'] = base_final['TRAF. REDES 5G MB']/1000
base_final['TRAF. BI TOTAL GB'] = base_final['TRAF. BI TOTAL MB']/1000
base_final['TRAF. BI 5G GB'] = base_final['TRAF. BI 5G MB']/1000

base_final = pd.merge(base_final, bi_merged_parteii, how='left', on='IBGE')

base_final['% SITES 5G'] = base_final['SITES 5G']/base_final['SITES']
base_final['RETENCAO 5G'] = (base_final['TRAF. REDES 5G GB']/base_final['TRAF. REDES TOTAL GB'])/(base_final['TRAF. BI 5G GB']/base_final['TRAF. BI TOTAL GB'])
base_final['% TRAF DESVIADO'] = base_final['TRAF DESVIADO']/base_final['TRAF. BI TOTAL GB']
base_final['% PARQUE DESVIADO'] = base_final['PARQUE DESVIADO']/base_final['PARQUE BI TOTAL']

base_final = pd.merge(base_final, bi_merged_parteiii, how='left' , on='IBGE')

base_final['ARGB'] = base_final['FATURAMENTO']/base_final['TRAF. BI TOTAL GB']
base_final['PERFIL'] = base_final['TRAF. BI TOTAL GB']/base_final['PARQUE BI TOTAL']
base_final['ARPU'] = base_final['FATURAMENTO']/base_final['PARQUE BI TOTAL']

In [ ]:
ano_atual = datetime.now().year
arquivo = f'0{mes_numero}{ano_atual}.xlsx'
arquivo = "042025.xlsx"
base_final.to_excel(arquivo, sheet_name='BASE', index=False)